In [33]:
#!pip install webdriver-manager --upgrade
#!pip install packaging
#!pip install -U selenium==4.11.2

  Attempting uninstall: webdriver-manager
    Found existing installation: webdriver-manager 3.8.5
    Uninstalling webdriver-manager-3.8.5:
      Successfully uninstalled webdriver-manager-3.8.5


In [37]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

driver = webdriver.Chrome()
driver.get("https://www.youtube.com/@ColeHastings/videos")

scroll_pause_time = 2  # Reduce the pause time for faster scrolling
scroll_increment = 3  # Number of scroll steps in one go

# Scroll to the bottom of the page
while True:
    for _ in range(scroll_increment):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)  # Short pause between each scroll step

    time.sleep(scroll_pause_time)
    
    new_height = driver.execute_script('return document.documentElement.scrollHeight')
    
    if new_height == last_height:
        break
    
    last_height = new_height

titles = driver.find_elements(By.ID, "video-title")
images = driver.find_elements(By.XPATH, '//a[@id="thumbnail"]/yt-image/img')
data = []

for i, j in zip(titles, images):
    data.append([i.text, j.get_attribute('src')])

df = pd.DataFrame(data, columns=['title', 'thumbnail'])

driver.quit()


In [38]:
df.head(20)

,title,thumbnail
0,The Evolution Of Masculinity: Why It’s So Hard...,https://i.ytimg.com/vi/KZX_ucn3Dvk/hqdefault.j...
1,5 Things I Wish I Knew At 20,https://i.ytimg.com/vi/98PEQwphc4Y/hqdefault.j...
2,How Modern Society Keeps You Submissive And Co...,https://i.ytimg.com/vi/dNIpeC-44wA/hqdefault.j...
3,Why Everyone Treats You Like A Ghost,https://i.ytimg.com/vi/wTxmTlMMuhU/hqdefault.j...
4,What 5 Hours Of Doomscrolling Does To Your Brain,https://i.ytimg.com/vi/Ps0MW3xTb3Y/hqdefault.j...
5,How To Stop Being A Slave To Meaningless Pleas...,https://i.ytimg.com/vi/qDUjclZBYEw/hqdefault.j...
6,"The Negative Cycle You're In, Ends Today",https://i.ytimg.com/vi/D4RfwsxDC-M/hqdefault.j...
7,Why You Fall In Love So Easily,https://i.ytimg.com/vi/qKwRmsEgfg8/hqdefault.j...
8,Stop Trying To Be A “High Value Man”,https://i.ytimg.com/vi/aVWdF1XSCD4/hqdefault.j...
9,How To Stop Wasting Your Life,https://i.ytimg.com/vi/Q6uk5NL0O8Q/hqdefault.j...


In [40]:
count = df['thumbnail'].isna().sum()
print(f'count none values:{count}')

count none values:0


In [43]:
cell_value = df.at[5, 'thumbnail']
print("Contents of cell:", cell_value)

Contents of cell: https://i.ytimg.com/vi/qDUjclZBYEw/hqdefault.jpg?sqp=-oaymwEcCNACELwBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLDkla9-MIq1ru8ectJsA7CINUa3rg


In [44]:
import os
import requests

# Create a folder to save the thumbnails
if not os.path.exists('thumbnails'):
    os.mkdir('thumbnails')

# Assuming you already have a DataFrame named 'df' with 'title' and 'thumbnail' columns
for index, row in df.iterrows():
    title = row['title']
    thumbnail_url = row['thumbnail']
    
    # Replace invalid characters in the title to create a valid filename
    filename = re.sub(r'[\/:*?"<>|]', '', title) + '.jpg'
    filepath = os.path.join('thumbnails', filename)
    
    # Download the thumbnail
    response = requests.get(thumbnail_url)
    if response.status_code == 200:
        with open(filepath, 'wb') as file:
            file.write(response.content)
